# Segmenting and Clustering Neighborhoods in Toronto

## 1. Scrape postal codes

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Extract tables
df_url = pd.read_html(url)

# Get first table                                                                                                           
df_table = df_url[0]


df_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code      180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


In [3]:
df_table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#drop rows with "Borough" = "Not assigned"
i_borough_na = df_table[df_table["Borough"] == "Not assigned"].index

df_table_1 = df_table.drop(i_borough_na, axis=0)
df_table_1.shape

(103, 3)

In [5]:
#if "Neighbourhood" = "Not assigned" take over "Borough"
def adapt_hood(hood, borough):
    if hood == "Not assigned":
        return borough
    else:
        return hood

df_table_1["Neighbourhood"] = df_table_1.apply(lambda x: adapt_hood(x["Neighbourhood"], x["Borough"]), axis=1)

In [6]:
#check for duplicate Postal Codes
df_group_pc = df_table_1.groupby(["Postal Code"])["Neighbourhood"].count().to_frame().sort_values(by="Neighbourhood", ascending=False)
df_group_pc.head(5)

,Neighbourhood
Postal Code,
M1B,1
M5R,1
M6G,1
M6E,1
M6C,1


In [7]:
df_table_1.shape

(103, 3)

## 2. Geocoding

In [8]:
! pip install geocoder

In [9]:
import geocoder # import geocoder

In [10]:
"""
loc_list = []

for index, row in df_table_1.iterrows():
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(row["Postal Code"]))
      lat_lng_coords = g.latlng
        
    loc_list.append([row["Postal Code"], lat_lng_coords[0], lat_lng_coords[1]])  
    
loc_list
"""

'\nloc_list = []\n\nfor index, row in df_table_1.iterrows():\n    # initialize your variable to None\n    lat_lng_coords = None\n    \n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n      g = geocoder.google(\'{}, Toronto, Ontario\'.format(row["Postal Code"]))\n      lat_lng_coords = g.latlng\n        \n    loc_list.append([row["Postal Code"], lat_lng_coords[0], lat_lng_coords[1]])  \n    \nloc_list\n'

Geocoder not accessible. Using csv-file instead

In [11]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv")
df_coord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [12]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_hoods_coord = pd.merge(df_table_1, df_coord, how="left", on="Postal Code")
df_hoods_coord.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
Postal Code      103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [14]:
df_hoods_coord.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 3. Venues in Toronto Neighborhoods

In [15]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [16]:
#filter for boroughs containing "Toronto"
df_toronto = df_hoods_coord[df_hoods_coord["Borough"].str.contains("Toronto")]
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [17]:
df_toronto.shape

(39, 5)

### 3.1 Foursquare credentials

In [18]:
credentials = pd.read_excel("../Credentials.xlsx", header=0)
credentials.columns

Index(['Provider', 'Key', 'Value'], dtype='object')

In [19]:
provider = "Foursquare"
cred_fsquare = credentials[credentials["Provider"] == provider]

CLIENT_ID = cred_fsquare[cred_fsquare["Key"] == "CLIENT_ID"].values[0][2] # your Foursquare ID
CLIENT_SECRET = cred_fsquare[cred_fsquare["Key"] == "CLIENT_SECRET"].values[0][2] # your Foursquare Secret
ACCESS_TOKEN = cred_fsquare[cred_fsquare["Key"] == "ACCESS_TOKEN"].values[0][2] # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### 3.2 Pull venues per hood
FUNCTION: to get TOP100 venues with 500m radius of hood

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
df_tor_venues = getNearbyVenues(df_toronto["Neighbourhood"], df_toronto["Latitude"], df_toronto["Longitude"], radius=500)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [22]:
df_tor_venues.shape

(1618, 7)

In [23]:
df_tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [24]:
df_tor_venues.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,63,63,63,63,63,63
Christie,15,15,15,15,15,15
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


### 3.3 Prepare features per hood

In [25]:
# one hot encoding
tor_onehot = pd.get_dummies(df_tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column
tor_onehot['Neighborhood'] = df_tor_venues['Neighborhood'] 

# where is column "Neighborhood"
index_new_col = tor_onehot.columns.get_loc("Neighborhood")
list_new = list(tor_onehot.columns)
list_new.remove("Neighborhood")

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[index_new_col]] + list_new
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# group the hoods with the main of venue categories
tor_vencat_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_vencat_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.016949,...,0.000000,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.015873,0.000000,0.000000,0.015873,0.000000,0.015873
5,Christie,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.000,0.000,0.000,0.012500,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.012500,0.000000,0.000000,0.025000
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,0.010000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
tor_vencat_grouped.shape

(39, 235)

Sorting for Top10-venues per hood

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_vencat_grouped['Neighborhood']

for ind in np.arange(tor_vencat_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_vencat_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Cheese Shop,Bakery,Restaurant,Seafood Restaurant,Farmers Market,Irish Pub,Butcher
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Grocery Store,Restaurant,Stadium,Intersection,Bar,Bakery,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Brewery,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Bar,Boat or Ferry,Boutique,Coffee Shop
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Salad Place,Bubble Tea Shop,Japanese Restaurant,Thai Restaurant,Ramen Restaurant


## 4. Clustering Toronto Neighbourhoods

In [30]:
# set number of clusters
kclusters = 5

tor_grouped_cluster = tor_vencat_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto.rename(columns={"Neighbourhood":"Neighborhood"}, inplace=False)

# merge  to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Café,Park,Breakfast Spot,Theater,Pub,Brewery,Shoe Store,French Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Bar,Mexican Restaurant,Diner,Beer Bar,Discount Store,Italian Restaurant,Japanese Restaurant,Portuguese Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Hotel,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Diner,Lingerie Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,American Restaurant,Cocktail Bar,Gastropub,Lingerie Store,Seafood Restaurant,Italian Restaurant,Restaurant,Cheese Shop
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant


In [32]:
latitude = 43.67
longitude = -79.4

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
# check appearance of clusters
toronto_merged.groupby(["Cluster Labels"])["Neighborhood"].count()

Cluster Labels
0     2
1    33
2     2
3     1
4     1
Name: Neighborhood, dtype: int64

Cluster 1 is the most common cluster.<br>
Let's see what is the difference between the clusters

In [40]:
#Cluster 0
toronto_merged.loc[toronto_merged["Cluster Labels"] == 0, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Christie,0,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Candy Store,Baby Store,Restaurant,Coffee Shop,Creperie
61,Lawrence Park,0,Park,Bus Line,Swim School,Business Service,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [41]:
#Cluster 1
toronto_merged.loc[toronto_merged["Cluster Labels"] == 1, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",1,Coffee Shop,Bakery,Café,Park,Breakfast Spot,Theater,Pub,Brewery,Shoe Store,French Restaurant
4,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Sushi Restaurant,Bar,Mexican Restaurant,Diner,Beer Bar,Discount Store,Italian Restaurant,Japanese Restaurant,Portuguese Restaurant
9,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Hotel,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Diner,Lingerie Store
15,St. James Town,1,Coffee Shop,Café,American Restaurant,Cocktail Bar,Gastropub,Lingerie Store,Seafood Restaurant,Italian Restaurant,Restaurant,Cheese Shop
19,The Beaches,1,Pub,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
20,Berczy Park,1,Coffee Shop,Cocktail Bar,Beer Bar,Cheese Shop,Bakery,Restaurant,Seafood Restaurant,Farmers Market,Irish Pub,Butcher
24,Central Bay Street,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Salad Place,Bubble Tea Shop,Japanese Restaurant,Thai Restaurant,Ramen Restaurant
30,"Richmond, Adelaide, King",1,Coffee Shop,Café,Restaurant,Deli / Bodega,Hotel,Thai Restaurant,Gym,Salad Place,Sushi Restaurant,Cosmetics Shop
31,"Dufferin, Dovercourt Village",1,Bakery,Pharmacy,Grocery Store,Pet Store,Pool,Music Venue,Middle Eastern Restaurant,Café,Brewery,Supermarket
36,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Aquarium,Café,Hotel,Brewery,Scenic Lookout,Italian Restaurant,Fried Chicken Joint,Restaurant,Music Venue


In [42]:
#Cluster 2
toronto_merged.loc[toronto_merged["Cluster Labels"] == 2, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,"Forest Hill North & West, Forest Hill Road Park",2,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
91,Rosedale,2,Park,Playground,Trail,Yoga Studio,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop


In [43]:
#Cluster 3
toronto_merged.loc[toronto_merged["Cluster Labels"] == 3, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,"Moore Park, Summerhill East",3,Restaurant,Trail,Tennis Court,Department Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop


In [44]:
#Cluster 4
toronto_merged.loc[toronto_merged["Cluster Labels"] == 4, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Roselawn,4,Garden,Health & Beauty Service,Pool,Home Service,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
